# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.57,75,36,12.44,SH,1724345929
1,1,codrington,-38.2667,141.9667,7.72,96,100,3.55,AU,1724345930
2,2,prineville,44.2999,-120.8345,16.66,61,40,2.06,US,1724345932
3,3,luanda,-8.8368,13.2343,24.00,69,0,5.66,AO,1724345858
4,4,waitangi,-43.9535,-176.5597,8.79,92,0,1.79,NZ,1724345934


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] < 27) & 
                                (city_data_df["Max Temp"] > 21) & 
                                (city_data_df["Wind Speed"] < 4.5) &
                                (city_data_df["Cloudiness"] == 0)].copy()

# Drop any rows with null values in place
ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,hamilton,39.1834,-84.5333,25.49,42,0,2.68,US,1724346070
155,155,bella vista,36.4295,-94.2316,25.47,58,0,3.60,US,1724346124
208,208,zabol,32.2500,67.2500,23.63,12,0,4.31,AF,1724346185
291,291,upington,-28.4478,21.2561,23.21,13,0,2.06,ZA,1724346211
314,314,kapuskasing,49.4169,-82.4331,23.00,56,0,2.06,CA,1724346313
316,316,greenville,35.6127,-77.3663,25.63,47,0,3.09,US,1724346049
329,329,hwange,-18.3693,26.5019,23.55,25,0,1.51,ZW,1724346332
358,358,newman,37.3138,-121.0208,21.69,63,0,0.45,US,1724346366
365,365,kill devil hills,36.0307,-75.6760,23.32,50,0,3.60,US,1724346374
392,392,kerman,30.2832,57.0788,26.94,13,0,4.47,IR,1724346406


### Step 3: Create a new DataFrame called `hotel_df`.

In [48]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,75,
1,codrington,AU,-38.2667,141.9667,96,
2,prineville,US,44.2999,-120.8345,61,
3,luanda,AO,-8.8368,13.2343,69,
4,waitangi,NZ,-43.9535,-176.5597,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "limit": 1,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
prineville - nearest hotel: Best Western
luanda - nearest hotel: Hotel Forum
waitangi - nearest hotel: Hotel Chathams
new plymouth - nearest hotel: Carrington Motel
adamstown - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
north bondi - nearest hotel: Beach Road Hotel
sao vicente - nearest hotel: Pousada Vitória
lebu - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
pacific grove - nearest hotel: Pacific Grove Inn
avarua - nearest hotel: Paradise Inn
kayangel - nearest hotel: No hotel found
pospelikha - nearest hotel: Транзит
anadyr - nearest hotel: Гостевой дом
tual - nearest hotel: Suita
vadso - nearest hotel: Scandic Vadsø
yellowknife - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Dali
port-aux-francais - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,75,No hotel found
1,codrington,AU,-38.2667,141.9667,96,No hotel found
2,prineville,US,44.2999,-120.8345,61,Best Western
3,luanda,AO,-8.8368,13.2343,69,Hotel Forum
4,waitangi,NZ,-43.9535,-176.5597,92,Hotel Chathams


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [52]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 1,
    size = "Humidity",
    color = "City",
    alpha=0.75,
    title="Humidity Map by City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)